In [1]:
import os
import time
import logging 
from custome_logger import setup_custom_logger
logger = setup_custom_logger()
logger.debug("test")

import torch
import torch.nn as nn
import torch.utils.benchmark as benchmark
from timeit import default_timer as timer
import warnings
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.runner import load_checkpoint, wrap_fp16_model
from os import path as osp
from mmdet3d.models import build_model
from mmdet3d.datasets import build_dataset
from mmcv.parallel import MMDataParallel
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)





In [2]:

device = torch.device("cuda:0")
cfg = Config.fromfile(
    r"/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny.py"
)

# if args.cfg_options is not None:
#     cfg.merge_from_dict(args.cfg_options)
# import modules from string list.
if cfg.get("custom_imports", None):
    from mmcv.utils import import_modules_from_strings

    import_modules_from_strings(**cfg["custom_imports"])

# import modules from plguin/xx, registry will be updated
if hasattr(cfg, "plugin"):
    if cfg.plugin:
        import importlib

        if hasattr(cfg, "plugin_dir"):
            plugin_dir = cfg.plugin_dir
            _module_dir = os.path.dirname(plugin_dir)
            _module_dir = _module_dir.split("/")
            _module_path = _module_dir[0]

            for m in _module_dir[1:]:
                _module_path = _module_path + "." + m
            print(_module_path)
            plg_lib = importlib.import_module(_module_path)
        else:
            # import dir is the dirpath for the config file
            _module_dir = os.path.dirname(
                r"/home/niklas/ETM_BEV/BEVerse/projects/configs/beverse_tiny.py"
            )
            _module_dir = _module_dir.split("/")
            _module_path = _module_dir[0]
            for m in _module_dir[1:]:
                _module_path = _module_path + "." + m
            print(_module_path)
            plg_lib = importlib.import_module(_module_path)
            

samples_per_gpu = 1
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    samples_per_gpu = cfg.data.test.pop("samples_per_gpu", 1)
    if samples_per_gpu > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    samples_per_gpu = max(
        [ds_cfg.pop("samples_per_gpu", 1) for ds_cfg in cfg.data.test]
    )
    if samples_per_gpu > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

torch.backends.cudnn.benchmark = True


@torch.no_grad()
def measure_time_host(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start = timer()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        if synchronize:
            torch.cuda.synchronize()
        end = timer()
        elapsed_time_ms = (end - start) * 1000

    else:
        for _ in range(num_repeats):
            start = timer()
            _ = model.forward(input_tensor)
            if synchronize:
                torch.cuda.synchronize()
            end = timer()
            elapsed_time_ms += (end - start) * 1000

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def measure_time_device(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        end_event.record()
        if synchronize:
            # This has to be synchronized to compute the elapsed time.
            # Otherwise, there will be runtime error.
            torch.cuda.synchronize()
        elapsed_time_ms = start_event.elapsed_time(end_event)

    else:
        for _ in range(num_repeats):
            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)
            start_event.record()
            _ = model.forward(input_tensor)
            end_event.record()
            if synchronize:
                # This has to be synchronized to compute the elapsed time.
                # Otherwise, there will be runtime error.
                torch.cuda.synchronize()
            elapsed_time_ms += start_event.elapsed_time(end_event)

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def run_inference(model: nn.Module, input_tensor: torch.Tensor) -> torch.Tensor:

    return model.forward(input_tensor)

projects.mmdet3d_plugin


In [3]:
!pwd

/home/niklas/ETM_BEV/BEVerse


In [4]:

torch.backends.cudnn.benchmark = True


@torch.no_grad()
def measure_time_host(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start = timer()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        if synchronize:
            torch.cuda.synchronize()
        end = timer()
        elapsed_time_ms = (end - start) * 1000

    else:
        for _ in range(num_repeats):
            start = timer()
            _ = model.forward(input_tensor)
            if synchronize:
                torch.cuda.synchronize()
            end = timer()
            elapsed_time_ms += (end - start) * 1000

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def measure_time_device(
    model: nn.Module,
    input_tensor: torch.Tensor,
    num_repeats: int = 100,
    num_warmups: int = 10,
    synchronize: bool = True,
    continuous_measure: bool = True,
) -> float:

    for _ in range(num_warmups):
        _ = model.forward(input_tensor)
    torch.cuda.synchronize()

    elapsed_time_ms = 0

    if continuous_measure:
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_repeats):
            _ = model.forward(input_tensor)
        end_event.record()
        if synchronize:
            # This has to be synchronized to compute the elapsed time.
            # Otherwise, there will be runtime error.
            torch.cuda.synchronize()
        elapsed_time_ms = start_event.elapsed_time(end_event)

    else:
        for _ in range(num_repeats):
            start_event = torch.cuda.Event(enable_timing=True)
            end_event = torch.cuda.Event(enable_timing=True)
            start_event.record()
            _ = model.forward(input_tensor)
            end_event.record()
            if synchronize:
                # This has to be synchronized to compute the elapsed time.
                # Otherwise, there will be runtime error.
                torch.cuda.synchronize()
            elapsed_time_ms += start_event.elapsed_time(end_event)

    return elapsed_time_ms / num_repeats


@torch.no_grad()
def run_inference(model: nn.Module, input_tensor: torch.Tensor) -> torch.Tensor:
    # from model.forward because BEVerse differentiates between different input types - img lidar etc 
    return model.forward_dummy()

In [5]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=samples_per_gpu,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)



In [6]:
sample = next(iter(data_loader))

In [7]:
sample.keys()

dict_keys(['img_metas', 'img_inputs', 'semantic_indices', 'semantic_map', 'future_egomotions', 'gt_bboxes_3d', 'gt_labels_3d', 'motion_segmentation', 'motion_instance', 'instance_centerness', 'instance_offset', 'instance_flow', 'has_invalid_frame', 'img_is_valid'])

In [8]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
wrap_fp16_model(model)
model.cuda()
model = MMDataParallel(model, device_ids=[0])


In [9]:
motion_distribution_targets = {
    # for motion prediction
    'motion_segmentation': sample['motion_segmentation'][0],
    'motion_instance': sample['motion_instance'][0],
    'instance_centerness': sample['instance_centerness'][0],
    'instance_offset': sample['instance_offset'][0],
    'instance_flow': sample['instance_flow'][0],
    'future_egomotion': sample['future_egomotions'][0],
}


with torch.no_grad():
    result = model(
        return_loss=False,
        rescale=True,
        img_metas=sample['img_metas'],
        img_inputs=sample['img_inputs'],
        future_egomotions=sample['future_egomotions'],
        motion_targets=motion_distribution_targets,
        img_is_valid=sample['img_is_valid'][0],
    )


Simple test
Started Logger
simple_test_pts


Simple test

batch?:  torch.Size([1, 3, 6, 3, 256, 704])

B 1, S 3, N 6, C 3, imH 256, imW 704

imgs  torch.Size([18, 3, 256, 704])

after backbone:  2

shape in list:  [torch.Size([18, 384, 16, 44]), torch.Size([18, 768, 8, 22])]

after backbone with_img_neck:  torch.Size([18, 512, 16, 44])

after transformation:  torch.Size([1, 3, 6, 512, 16, 44])

after LLS :  torch.Size([1, 3, 64, 128, 128])

after Temporal :  torch.Size([1, 64, 128, 128])

simple_test_pts

In [20]:
result["time_stats"]

{'t_BEV': 1663055717.0110373,
 't_temporal': 1663055717.5791538,
 't0': 1663055712.6830127,
 't_end': 1663055746.884341}

x, rots, trans, intrins, post_rots, post_trans = input

'CAM_FRONT_RIGHT': {
    'data_path': './data/nuscenes/samples/CAM_FRONT_RIGHT/n015-2018-10-08-15-36-50+0800__CAM_FRONT_RIGHT__1538984233520339.jpg', 
    'type': 'CAM_FRONT_RIGHT', 
    'sample_data_token': 'd6f89460954c43d39ed7c9ac91ab03d0', 
    'sensor2ego_translation': [1.5508477543, -0.493404796419, 1.49574800619], 
    'sensor2ego_rotation': [0.2060347966337182, -0.2026940577919598, 0.6824507824531167, -0.6713610884174485], 'ego2global_translation': [715.6566537856895, 1810.1516804263824, 0.0], 
    'ego2global_rotation': [0.8004835927391405, 0.00541081062084495, -0.0028680900818508943, -0.5993233809414961], 'timestamp': 1538984233520339, 
    'sensor2lidar_rotation': array([[ 0.55971752, -0.01057234,  0.82861603],
       [-0.82828535,  0.02385392,  0.55979851],
       [-0.02568412, -0.99965955,  0.00459454]]), 
    'sensor2lidar_translation': array([ 0.48135698,  0.51094805, -0.32997566]), 
    'cam_intrinsic': array([[1.26084744e+03, 0.00000000e+00, 8.07968245e+02],
       [0.00000000e+00, 1.26084744e+03, 4.95334427e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])},

In [19]:
sample["img_metas"][0]

DataContainer([[{'flip': False, 'pcd_horizontal_flip': False, 'pcd_vertical_flip': False, 'box_mode_3d': <Box3DMode.LIDAR: 0>, 'box_type_3d': <class 'mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes'>, 'sample_idx': 'b5989651183643369174912bc5641d3b', 'pcd_scale_factor': 1.0, 'pts_filename': './data/nuscenes/samples/LIDAR_TOP/n015-2018-10-08-15-36-50+0800__LIDAR_TOP__1538984233547259.pcd.bin', 'img_info': [{'CAM_FRONT': {'data_path': './data/nuscenes/samples/CAM_FRONT/n015-2018-10-08-15-36-50+0800__CAM_FRONT__1538984233512470.jpg', 'type': 'CAM_FRONT', 'sample_data_token': '524d443c501a4f98a14508c3fb6f6de3', 'sensor2ego_translation': [1.70079118954, 0.0159456324149, 1.51095763913], 'sensor2ego_rotation': [0.4998015430569128, -0.5030316162024876, 0.4997798114386805, -0.49737083824542755], 'ego2global_translation': [715.6477969391926, 1810.182287996884, 0.0], 'ego2global_rotation': [0.8012964074773921, 0.00544170305513258, -0.0028663044812231933, -0.5982359396845913], 'times

In [6]:
batch_size = 1
num_warmups = 100
num_repeats = 1000
# Change to C x 1 x 3 x 1600 x 900
# 704×256 Tiny
# 1408×512
# B, S, N, C, imH, imW = imgs.shape
img_inputs =  torch.rand(1,7,6,3,704,256).cuda()
# (batch, seq, num_cam)
future_egomotions = torch.zeros((batch_size, 7, 6)).type_as(img_inputs).cuda()
img_is_valid = torch.ones((batch_size, 7)).type_as(img_inputs) > 0
img_is_valid.cuda()
print("Future Egomotions: ", future_egomotions.shape)
print("img_is_valid: ", img_is_valid.shape)


Future Egomotions:  torch.Size([1, 7, 6])
img_is_valid:  torch.Size([1, 7])


In [7]:
# #single frame 
# future_egomotions = future_egomotions[:, :1]
# img_is_valid = img_is_valid[:, :1]
# print("Future Egomotions: ", future_egomotions.shape)
# print("img_is_valid: ", img_is_valid.shape)


In [8]:
from mmdet3d.core.bbox.structures.box_3d_mode import LiDARInstance3DBoxes

dummy_lidar2ego_rots = (
    torch.tensor(
        [
            [-5.4280e-04, 9.9893e-01, 4.6229e-02],
            [-1.0000e00, -4.0569e-04, -2.9750e-03],
            [-2.9531e-03, -4.6231e-02, 9.9893e-01],
        ]
    )
    .type_as(img_inputs)
    .cpu()
)
dummy_lidar2ego_trans = (
    torch.tensor([0.9858, 0.0000, 1.8402]).type_as(img_inputs).cpu()
)
print(dummy_lidar2ego_rots.shape)
print(dummy_lidar2ego_trans.shape)
img_metas = [
    dict(
        box_type_3d=LiDARInstance3DBoxes,
        lidar2ego_rots=dummy_lidar2ego_rots,
        lidar2ego_trans=dummy_lidar2ego_trans,
    )
]
print("image meta:", img_metas)

torch.Size([3, 3])
torch.Size([3])
image meta: [{'box_type_3d': <class 'mmdet3d.core.bbox.structures.lidar_box3d.LiDARInstance3DBoxes'>, 'lidar2ego_rots': tensor([[-5.4280e-04,  9.9893e-01,  4.6229e-02],
        [-1.0000e+00, -4.0569e-04, -2.9750e-03],
        [-2.9531e-03, -4.6231e-02,  9.9893e-01]]), 'lidar2ego_trans': tensor([0.9858, 0.0000, 1.8402])}]


In [20]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
print(type(model))


<class 'projects.mmdet3d_plugin.models.detectors.beverse.BEVerse'>


In [9]:
model = build_model(cfg.model, test_cfg=cfg.get("test_cfg")).cuda()
wrap_fp16_model(model)
img_feats,time_dict  = model.extract_img_feat(
            img=img_inputs,
            img_metas=img_metas,
            future_egomotion=future_egomotions,
            img_is_valid=img_is_valid,
        )
print(img_feats.shape)
print(time_dict)

batch?:  torch.Size([1, 7, 6, 3, 704, 256])
B 1, S 7, 6, C 3, imH 704, imW 256
imgs  torch.Size([42, 3, 704, 256])
after backbone:  2
shape in list:  [torch.Size([42, 384, 44, 16]), torch.Size([42, 768, 22, 8])]
after backbone with_img_neck:  torch.Size([42, 512, 44, 16])
after transformation:  torch.Size([1, 7, 6, 512, 44, 16])


TypeError: can only concatenate list (not "Tensor") to list

In [13]:
img_inputs =  torch.rand(1,7,6,3,704,256)
print(img_inputs[1:].shape)
trans_output = torch.rand(1, 7, 6, 512, 44, 16)
img_inputs =  torch.rand(2,7,6,3,704,256)
print(img_inputs[1:].shape)
test = [trans_output] + img_inputs[1:]
test.shape

torch.Size([0, 7, 6, 3, 704, 256])
torch.Size([1, 7, 6, 3, 704, 256])


TypeError: can only concatenate list (not "Tensor") to list

In [16]:

num_warmups = 100
num_repeats = 1000
# Change to C x 1 x 3 x 1600 x 900
# 704×256 Tiny
# 1408×512
# B, T, N, C, imH, imW = imgs.shape
input_shape = (1,3,6,3,1600,900)

future_egomotions = torch.zeros((batch_size, 7, 6)).type_as(img_inputs)
img_is_valid = torch.ones((batch_size, 7)).type_as(img_inputs) > 0
print("Future Egomotions: ", future_egomotions.shape)
print("img_is_valid: ", img_is_valid.shape)


# imgs = imgs.view(B * S * N, C, imH, imW)

model = build_model(cfg.model, test_cfg=cfg.get("test_cfg"))
wrap_fp16_model(model)
# load_checkpoint(
#     model,
#     r"/home/niklas/ETM_BEV/BEVerse/checkpoints/beverse_tiny.pth",
#     map_location="cpu",
# )
# model = fuse_module(model)
model.cuda(device)
model.eval()
# model = nn.Conv2d(in_channels=input_shape[1], out_channels=256, kernel_size=(5, 5))

# Input tensor
input_tensor = torch.rand(input_shape, device=device)

torch.cuda.synchronize()

print("Latency Measurement Using CPU Timer...")
for continuous_measure in [True]:
    for synchronize in [True]:
        try:
            latency_ms = measure_time_host(
                model=model,
                input_tensor=input_tensor,
                num_repeats=num_repeats,
                num_warmups=num_warmups,
                synchronize=synchronize,
                continuous_measure=continuous_measure,
            )
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: {latency_ms:.5f} ms| "
            )
        except Exception as e:
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: N/A     ms| "
            )
        torch.cuda.synchronize()

print("Latency Measurement Using CUDA Timer...")
for continuous_measure in [True, False]:
    for synchronize in [True, False]:
        try:
            latency_ms = measure_time_device(
                model=model,
                input_tensor=input_tensor,
                num_repeats=num_repeats,
                num_warmups=num_warmups,
                synchronize=synchronize,
                continuous_measure=continuous_measure,
            )
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: {latency_ms:.5f} ms| "
            )
        except Exception as e:
            print(
                f"|"
                f"Synchronization: {synchronize!s:5}| "
                f"Continuous Measurement: {continuous_measure!s:5}| "
                f"Latency: N/A     ms| "
            )
        torch.cuda.synchronize()

print("Latency Measurement Using PyTorch Benchmark...")
num_threads = 1
timer = benchmark.Timer(
    stmt="run_inference(model, input_tensor)",
    setup="from __main__ import run_inference",
    globals={"model": model, "input_tensor": input_tensor},
    num_threads=num_threads,
    label="Latency Measurement",
    sub_label="torch.utils.benchmark.",
)

profile_result = timer.timeit(num_repeats)
# https://pytorch.org/docs/stable/_modules/torch/utils/benchmark/utils/common.html#Measurement
print(f"Latency: {profile_result.mean * 1000:.5f} ms")


Latency Measurement Using CPU Timer...
|Synchronization: True | Continuous Measurement: True | Latency: N/A     ms| 
Latency Measurement Using CUDA Timer...
|Synchronization: True | Continuous Measurement: True | Latency: N/A     ms| 
|Synchronization: False| Continuous Measurement: True | Latency: N/A     ms| 
|Synchronization: True | Continuous Measurement: False| Latency: N/A     ms| 
|Synchronization: False| Continuous Measurement: False| Latency: N/A     ms| 
Latency Measurement Using PyTorch Benchmark...


FileNotFoundError: [Errno 2] No such file or directory: 'dummy_input.pt'